# How to Automate Cold-Email Outreach with LLM Agents

Why Automate Cold Emails?
<br>Sending effective, personalized cold emails is time-consuming:
- You want variants of professional, witty, or concise emails to A/B test.
- You must pick the best draft, not just any draft.
- You need compelling subject lines.
- You have to convert text to HTML so it renders nicely.
- Finally, you still need to deliver via an Email Service Provider like SendGrid.

A modern agent framework lets each of these steps become an agent or tool. We simply orchestrate them.

Install these packages:
<br>`pip install sendgrid openai-agents`

- sendgrid – handles SMTP-class deliverability with minimal fuss.
- openai-agents – lets us compose “agents” (LLM instances + instructions + tools) into directed workflows.

We then load API keys (OPENAI_API_KEY, SENDGRID_API_KEY) and email addresses (SENDER_EMAIL, RECEIVER_EMAIL) from a .env file, keeping secrets out of source control.

In [17]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

load_dotenv(override=True)

True

In [18]:
# Configuration & credentials
llm_model = "gpt-4o-mini"
sender_email = os.environ["SENDER_EMAIL"]
receiver_email = os.environ["RECEIVER_EMAIL"]

### Prompt instructions for agents

Three distinct prompt templates:
- Professional Tone, Use-case: Formal, serious	Enterprise / C-suite
- Witty, Use-case: Humorous, disarming	Startups / creative directors
- Busy Tone, Use-case: Ultra-concise	Hard-pressed execs

Creating diversity up-front gives the Sales Manager real choices.

In [19]:
prof_instructions = "You are a sales agent working for idare.ai, \
a company that provides zero-code predictive analytics solution powered by AI. \
You write professional, serious cold emails."

witty_instructions = "You are a humorous, engaging sales agent working for idare.ai, \
a company that provides zero-code predictive analytics solution, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

busy_instructions = "You are a busy sales agent working for idare.ai, \
a company that provides provides zero-code predictive analytics solution, powered by AI. \
You write concise, to the point cold emails."

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

sales_manager_instructions = """
You are a Sales Manager at idare.ai. Your goal is to find the single best cold sales email using the sales tools.

Follow these steps carefully:
1. Generate Drafts: Use all three sales tools (prof_sales, witty_sales, busy_sales) to generate three different email drafts. Do not proceed until all three drafts are ready.

2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.

3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.

Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

email_manager_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

prompt = "Send out a cold sales email addressed to Dear CTO Dr. Khairul Chowdhury from Kawsar, Data Scientist at idare.ai"

### Turning Prompts into Agents
An Agent is basically LLM + system prompt with optional tools.

Each agent here has no extra tools – it just writes an email when called.

Subject lines and HTML formatting are specialized skills. Small, focused prompts make the LLM sharper at these micro-tasks.

subject_writer: turns the email body into an irresistible subject.

html_converter: converts Markdown-ish text into responsive HTML.

Both are exposed as tools so downstream agents can call them.

In [20]:
prof_sales_agent = Agent(name="Professional Sales Agent",
                         instructions=prof_instructions,
                         model=llm_model)

witty_sales_agent = Agent(name="Engaging Sales Agent",
                          instructions=witty_instructions,
                          model=llm_model)

busy_sales_agent = Agent(name="Busy Sales Agent",
                         instructions=busy_instructions,
                         model=llm_model)

subject_writer = Agent(name="Email subject writer",
                       instructions=subject_instructions,
                       model=llm_model)

html_converter = Agent(name="HTML email body converter",
                       instructions=html_instructions,
                       model=llm_model)

### Convert Agents into Tools

In [21]:
sales_agent_description = "Write a cold sales email"
prof_sales = prof_sales_agent.as_tool(tool_name="prof_sales_agent",
                                      tool_description=sales_agent_description)
witty_sales = witty_sales_agent.as_tool(tool_name="witty_sales_agent",
                                        tool_description=sales_agent_description)
busy_sales = busy_sales_agent.as_tool(tool_name="busy_sales_agent",
                                      tool_description=sales_agent_description)

subject_tool = subject_writer.as_tool(tool_name="subject_writer",
                                      tool_description="Write a subject for a cold sales email")

html_tool = html_converter.as_tool(tool_name="html_converter",
                                   tool_description="Convert a text email body to an HTML email body")

Everything so far has been pure text. We still need to actually send.

A small Python function wrapped by @function_tool lets agents trigger real I/O.

If SendGrid replies with an error, the agent chain can propagate it.

In [22]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        from_email = Email(sender_email)  # Change to your verified sender
        to_email = To(receiver_email)  # Change to your recipient
        content = Content("text/html", html_body)
        mail = Mail(from_email, to_email, subject, content).get()
        sg.client.mail.send.post(request_body=mail)
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

The Email Manager Agent
<br>Role: format and deliver the winning draft.

Its toolbox:
- `subject_writer`
- `html_converter`
- `send_html_email`

Its instruction set forces the exact ordering:
- Generate subject
- Convert to HTML
- Send via SendGrid

Because each tool is explicit, you get reliable, auditable steps.

The Sales Manager Agent
<br>This is the orchestrator:
- Calls all three drafting tools in parallel.
- Evaluates and picks one draft.
- Hands that draft off to the Email Manager.

The “handoff” mechanic in openai-agents is powerful: the Sales Manager doesn’t need to know how to send email – it merely passes control.

In [23]:
emailer_agent = Agent(name="Email Manager",
                      instructions=email_manager_instructions,
                      tools=[subject_tool, html_tool, send_html_email],
                      model=llm_model,
                      handoff_description="Convert an email to HTML and send it")

sales_manager = Agent(name="Sales Manager",
                      instructions=sales_manager_instructions,
                      tools=[prof_sales, witty_sales, busy_sales],
                      handoffs=[emailer_agent],
                      model=llm_model)

trace() logs each tool call and LLM message, invaluable for debugging and analytics (e.g., see which persona wins most often).

Runner.run() kicks off the graph:

User prompt → Sales Manager
   <br>&emsp;├─▶ prof_sales_agent
   <br>&emsp;├─▶ witty_sales_agent
   <br>&emsp;└─▶ busy_sales_agent
<br>Sales Manager picks a winner
   <br>&emsp;└─▶ Email Manager
         <br>&emsp;&emsp;&emsp;&emsp;&emsp;├─▶ subject_writer
         <br>&emsp;&emsp;&emsp;&emsp;&emsp;├─▶ html_converter
         <br>&emsp;&emsp;&emsp;&emsp;&emsp;└─▶ send_html_email

Finally the program prints the result (success or error)


In [24]:
with trace("Automated SDR"):
    result = await Runner.run(sales_manager, prompt)
    print(result)

RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    The cold sales email has been successfully sent to Dr. Khairul Chowdhury. If you need anything else, feel free to ask!
- 16 new item(s)
- 5 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


#### Now go and look at the trace

https://platform.openai.com/traces

By decomposing each micro-step of outbound outreach into a specialized agent or tool, we gain:

Modularity – swap out, retrain, or fine-tune individual parts.

Observability – inspect traces to debug failure points.

Scalability – run thousands of emails in parallel with minimal extra code.

In short, LLM agents plus a reliable ESP turn the cold emailing into a fully-automated, measurable pipeline—freeing you to focus on higher-level strategy.